### Synopsis
This code averages and reduces g2 for temperature ramping (up and down) of protein H02

In [1]:

import numpy as np
import matplotlib.pyplot as plt
import h5py
import pickle

import sys

sys.path.append('../common')
from utils import process_group

# mount_point = '/gdata/s8id-dmdtn/'
# mount_point = "/home/beams/8IDIUSER/Documents/llps-saxpcs/reanalysis_2025_0311/results/"
mount_point = "/home/beams/8IDIUSER/Documents/llps-saxpcs/reanalysis_2025_0428/results/2025_04_analysis/"

fpath1 = mount_point + '2021-2/cluster_results_all'

coef_sam = 6.93e4  # Absolute scattering cross-section coefficient for the samples
coef_buf = 7.62e4  # Absolute scattering cross-section coefficient for the buffer


### Averaging of SA-XPCS results, temperature ramp up

In [2]:
avg_ramp_up, t_el, ql_dyn, ql_sta = process_group(group='B039',
                                              num_sections=10,
                                              prefix=fpath1, 
                                              zone_idx='auto',
                                              num_cores=24)


index	 T-min(C)	 T-max(C)	 T-mean(C)	 points
n=00	    20.03	    22.52	  21.2783	 278
n=01	    22.54	    25.04	  23.7871	 278
n=02	    25.05	    27.55	  26.2965	 278
n=03	    27.55	    30.06	  28.8104	 278
n=04	    30.07	    32.58	  31.3258	 278
n=05	    32.59	     35.1	  33.8523	 278
n=06	    35.12	    37.64	  36.3797	 278
n=07	    37.65	    40.19	  38.9117	 278
n=08	     40.2	    42.71	  41.4595	 278
n=09	    42.73	    45.23	  43.9819	 278


### Averaging of SA-XPCS results, temperature ramp down

In [3]:

avg_ramp_down, _, _, _ = process_group(group='B040',
                                 num_sections=10, 
                                 prefix=fpath1,
                                 zone_idx='auto',
                                 num_cores=24)


index	 T-min(C)	 T-max(C)	 T-mean(C)	 points
n=00	    42.72	    45.23	  43.9767	 278
n=01	    40.18	    42.71	  41.4464	 278
n=02	    37.65	    40.17	  38.9206	 278
n=03	    35.11	    37.65	  36.3813	 278
n=04	    32.56	     35.1	  33.8364	 278
n=05	    30.06	    32.55	  31.3012	 278
n=06	    27.55	    30.05	  28.7991	 278
n=07	    25.05	    27.54	  26.2949	 278
n=08	    22.55	    25.04	  23.7972	 278
n=09	    20.05	    22.54	  21.2994	 278


### Averaging of background 

In [4]:

avg_bg, _, _, _ = process_group(group='D029',
                                 num_sections=1, 
                                 prefix=fpath1,
                                 zone_idx='auto',
                                 num_cores=24)

index	 T-min(C)	 T-max(C)	 T-mean(C)	 points
n=00	    24.98	    25.02	  25.0006	 200


### Data reduction, full result

Subtracting I(Q) with background, converting to absolute scattering cross-section, and dump to pickle file for plotting

In [5]:

for ii in range(len(avg_ramp_up)):
    avg_ramp_up[ii]["saxs_1d"] = coef_sam*avg_ramp_up[ii]["saxs_1d"] - coef_buf*avg_bg[0]["saxs_1d"]

for ii in range(len(avg_ramp_down)):
    avg_ramp_down[ii]["saxs_1d"] = coef_sam*avg_ramp_down[ii]["saxs_1d"] - coef_buf*avg_bg[0]["saxs_1d"]

with open('checkpoint_both.pickle', 'wb') as f:  # 'wb' for binary write mode
    pickle.dump(avg_ramp_up, f)  # Dump the object to the file
    pickle.dump(avg_ramp_down, f)
    pickle.dump(t_el, f)
    pickle.dump(ql_sta, f)
    pickle.dump(ql_dyn, f)
